In [1]:
# Import Packages

import datasets
import torch
import optuna

from datetime import datetime
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments

In [2]:
torch.cuda.is_available()

True

# Load the dataset

In [3]:
dataset = datasets.load_from_disk("data")
dataset

DatasetDict({
    train: Dataset({
        features: ['selftext', 'label'],
        num_rows: 3984
    })
    test: Dataset({
        features: ['selftext', 'label'],
        num_rows: 498
    })
    valid: Dataset({
        features: ['selftext', 'label'],
        num_rows: 498
    })
})

# Prepare Model and Dataset

In [4]:
# Define 4096 as our maximum sentence length

MAX_LEN = 4096

In [5]:
# Import tokenizer

tokenizer = LongformerTokenizer.from_pretrained('AIMH/mental-longformer-base-4096', max_length = MAX_LEN)

c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Define tokenization function

def tokenization(text):
    return tokenizer(text["selftext"], padding='max_length', truncation=True, max_length=MAX_LEN)

In [7]:
# Tokenize data

dataset["train"] = dataset["train"].map(tokenization)
dataset["valid"] = dataset["valid"].map(tokenization)
dataset["test"] = dataset["test"].map(tokenization)

In [8]:
# Sanity check: make sure our tokenization follows our max sentence length

len(dataset["valid"][4]["input_ids"])

4096

In [9]:
# Convert to Pytorch Tensor

dataset["train"].set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
dataset["valid"].set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
dataset["test"].set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Setting Up Hyperparameter Search

We will use HuggingFace Trainer API's [built-in hyperparameter testing feature](https://huggingface.co/docs/transformers/hpo_train) using Optuna backend

In [10]:
# Set up hyperparameter search space, compute metrics, and compute objective

def optuna_hp_space(trial: optuna.Trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-3, log=True),
        "warmup_steps": trial.suggest_int("warmup_steps", 50, 250, step=25),
        "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True),
    }

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, weighted_f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    macro_f1 = f1_score(labels, preds, average="macro")
    class_f1 = f1_score(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'macro_f1': macro_f1,
        # 'accuracy': acc,
        # 'weighted_f1': weighted_f1,
        # 'class_f1': class_f1,
        # 'precision': precision,
        # 'recall': recall
    }

def objective(metrics):
    print("Metrics:", metrics)
    return metrics["eval_macro_f1"]

In [11]:
# Set up other training arguments

training_args = TrainingArguments(
    output_dir = "frames",
    num_train_epochs = 5,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    per_device_eval_batch_size = 16,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    disable_tqdm = True, 
    load_best_model_at_end=True,
    logging_strategy = "epoch",
    fp16 = True,
    dataloader_num_workers = 0,
)

In [12]:
# Set up model init, trainer, and other training arguments

def model_init(trial):
    return LongformerForSequenceClassification.from_pretrained(
        'AIMH/mental-longformer-base-4096', 
        gradient_checkpointing=True, 
        attention_window=512, 
        num_labels=6,
    )

trainer = Trainer(
    model=None,
    model_init=model_init,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = dataset["train"],
    eval_dataset = dataset["valid"],
)

c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Hyperparameter search

best_trial = trainer.hyperparameter_search(
    # compute_objective=objective,
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20,
    study_name="MentalLongformer-" + datetime.now().strftime("%d-%m-%Y-%H-%M-%S"),
    storage="sqlite:///optuna/optuna.db",
    sampler=optuna.samplers.RandomSampler(),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=124, interval_steps=24
    ),
)

[I 2024-10-16 22:09:38,565] A new study created in RDB with name: MentalLongformer-16-10-2024-22-09-37
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Initializing global attention on CLS token...
c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


{'loss': 1.722, 'grad_norm': 1.5232858657836914, 'learning_rate': 0.0010899623215575313, 'epoch': 0.9959839357429718}


c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.7097824811935425, 'eval_macro_f1': 0.07067510548523206, 'eval_runtime': 81.4369, 'eval_samples_per_second': 6.115, 'eval_steps_per_second': 0.393, 'epoch': 0.9959839357429718}


c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


...

{'train_runtime': 18315.5556, 'train_samples_per_second': 1.088, 'train_steps_per_second': 0.034, 'train_loss': 0.8813935064500378, 'epoch': 4.979919678714859}


c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-d

{'loss': 1.4753, 'grad_norm': 6.962917804718018, 'learning_rate': 0.00011099180685069985, 'epoch': 0.9959839357429718}
{'eval_loss': 1.2829155921936035, 'eval_macro_f1': 0.42417051864609095, 'eval_runtime': 82.1667, 'eval_samples_per_second': 6.061, 'eval_steps_per_second': 0.389, 'epoch': 0.9959839357429718}


c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
[W 2024-10-18 09:31:36,481] Trial 9 failed with parameters: {'learning_rate': 0.00012146693721612734, 'warmup_steps': 75, 'weight_decay': 0.0015260719324010359} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\Server\Documents\Stackofsugar\ta\.venv\Lib\site-packages\transformers\integrations\integration_utils.py", line 247, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=tri

KeyboardInterrupt: 